# Demo of Computing Volume From Freesurfer Stats Files

In [1]:
import numpy as np
import os
import sys
import pandas as pd
import numpy as np
import nibabel as nb

In [2]:
# data directory of stats files
patid = 'la03'
datadir = os.path.join("/home/adam2392/hdd/data/neuroimaging/freesurfer_output/", patid)
# datadir = os.path.join("/Users/adam2392/Dropbox/phd_research/data/neuroimaging_results/freesurfer_output/", patid)

statsdir = os.path.join(datadir, "stats")

In [20]:
# extract arguments from parser
lh_atlas_stats_file = os.path.join(statsdir, "lh.aparc.stats")
rh_atlas_stats_file = os.path.join(statsdir, "rh.aparc.stats")
wm_stats_file = os.path.join(statsdir, "wmparc.stats")
subcort_stats_file = os.path.join(statsdir, "aseg.stats")

trglutfile = os.path.join("/Users/adam2392/Documents/neuroimg_pipeline/data/mrtrix3_labelconvert", "fs_default.txt")
srclutfile = os.path.join("/Users/adam2392/Documents/neuroimg_pipeline/data/")

trglutfile = os.path.join("/home/adam2392/Documents/neuroimg_pipeline/data/mrtrix3_labelconvert/fs_default.txt")
srclutfile = os.path.join("/home/adam2392/Documents/neuroimg_pipeline/data/FreeSurferColorLUT.txt")
                          
outdir = "./"

# Read in Files

In [5]:
table = np.genfromtxt(os.fspath(trglutfile), encoding="latin1", dtype=None)
if len(table.dtype) == 6:
    # id name R G B A
    inds = table[table.dtype.names[0]]
    names = table[table.dtype.names[1]].astype('U')
    r = table[table.dtype.names[2]]
    g = table[table.dtype.names[3]]
    b = table[table.dtype.names[4]]
    a = table[table.dtype.names[5]]
    shortnames = np.zeros(len(self.names), dtype='U')

elif len(table.dtype) == 7:
    # id shortname name R G B A
    inds = table[table.dtype.names[0]]
    shortnames = table[table.dtype.names[1]].astype('U')
    names = table[table.dtype.names[2]].astype('U')
    r = table[table.dtype.names[3]]
    g = table[table.dtype.names[4]]
    b = table[table.dtype.names[5]]
    a = table[table.dtype.names[6]]

In [10]:
# print(table)

In [59]:
class FSCorticalStats():
    """
          
    # StructName NumVert SurfArea GrayVol ThickAvg ThickStd MeanCurv GausCurv FoldInd CurvInd
    """
    def __init__(self, filepath):
        self.table = np.genfromtxt(filepath, encoding="latin1", dtype=None)
    
    @property
    def numverts(self):
        return self.table[table.dtype.names[1]]
    
    @property
    def names(self):
        return self.table[table.dtype.names[0]].astype('U')
    
    @property
    def surfareas(self):
        return self.table[table.dtype.names[2]]
    
    @property
    def volumes(self):
        return self.table[table.dtype.names[3]]
    
    def set_table_components(self):
        self.name = table[table.dtype.names[0]]
            
    def create_summary_stats(self, inds, statsobj):
        volumes = {}

        # for each of the indices, create metric measurements lists
        for ind in inds:
            volumes[statsobj.names[ind]] = statsobj.volumes[ind]

        return volumes
    
class FSSegmentationStats():
    """
     Index SegId NVoxels Volume_mm3 StructName normMean normStdDev normMin normMax normRange  
    """
    def __init__(self, filepath):
        self.table = np.genfromtxt(filepath, encoding="latin1", dtype=None)
    
    @property
    def indices(self):
        # begin indices at 0
        return self.table[table.dtype.names[0]] - 1
    
    @property
    def segindices(self):
        # begin indices at 0
        return self.table[table.dtype.names[1]] 
    
    @property
    def numvoxels(self):
        return self.table[table.dtype.names[2]]
    
    @property
    def volumes(self):
        return self.table[table.dtype.names[3]]
    
    @property
    def names(self):
        return self.table[table.dtype.names[4]].astype('U')

In [60]:
lhstats_df = FSCorticalStats(lh_atlas_stats_file)
rhstats_df = FSCorticalStats(rh_atlas_stats_file)
subcortstats_df = FSSegmentationStats(subcort_stats_file)
wmstats_df = FSSegmentationStats(wm_stats_file)
print(len(lhstats_df.names), len(rhstats_df.names), len(subcortstats_df.names))
print(34+34+45)
print(len(wmstats_df.names))

34 34 45
113
70


# Map from FS LUT -> MRTRIx3 LUT

Since most of our data for say a desikan atlas is in the MRTrix3 LUT index system (0-84), then we want to assign that corresponding index to each structure name in the computed volume statistics.

In [65]:
def create_volume_dict(fsobjs, ordering=[]):
    volumes = {}

    for j, obj in enumerate(fsobjs):
        if ordering:
            currorder = ordering[j]
        else:
            currorder = ''
            
        for idx, name in enumerate(obj.names):
            volumes[currorder+name] = obj.volumes[idx]

    return volumes

In [67]:
fsvolumedict = create_volume_dict([lhstats_df, rhstats_df, subcortstats_df], ordering=['lh-', 'rh-', ''])

print(fsvolumedict)

{'lh-bankssts': 2076, 'lh-caudalanteriorcingulate': 1196, 'lh-caudalmiddlefrontal': 6618, 'lh-cuneus': 3582, 'lh-entorhinal': 2594, 'lh-fusiform': 10682, 'lh-inferiorparietal': 13833, 'lh-inferiortemporal': 15346, 'lh-isthmuscingulate': 3014, 'lh-lateraloccipital': 14637, 'lh-lateralorbitofrontal': 8638, 'lh-lingual': 9801, 'lh-medialorbitofrontal': 5807, 'lh-middletemporal': 13020, 'lh-parahippocampal': 1821, 'lh-paracentral': 5331, 'lh-parsopercularis': 5441, 'lh-parsorbitalis': 2257, 'lh-parstriangularis': 3848, 'lh-pericalcarine': 2575, 'lh-postcentral': 11390, 'lh-posteriorcingulate': 2979, 'lh-precentral': 14113, 'lh-precuneus': 11331, 'lh-rostralanteriorcingulate': 2307, 'lh-rostralmiddlefrontal': 15982, 'lh-superiorfrontal': 22208, 'lh-superiorparietal': 14052, 'lh-superiortemporal': 10212, 'lh-supramarginal': 12817, 'lh-frontalpole': 988, 'lh-temporalpole': 2601, 'lh-transversetemporal': 692, 'lh-insula': 7688, 'rh-bankssts': 2691, 'rh-caudalanteriorcingulate': 1340, 'rh-cauda

In [30]:
class RegionIndexMapping(object):
    """
    Class wrapper for a region index mapping.

    This maps each index of the source file to an index in the corresponding look up table.
    """

    def __init__(self, color_lut_src_file: os.PathLike, color_lut_trg_file: os.PathLike):
        """
        :param color_lut_src_file: The source lookup table
        :param color_lut_trg_file: The target's lookup table.
        """
        self.src_table = ColorLut(color_lut_src_file)
        self.trg_table = ColorLut(color_lut_trg_file)

        names_to_trg = dict(zip(self.trg_table.names, self.trg_table.inds))

        self.src_to_trg = dict()
        for src_ind, src_name in zip(self.src_table.inds, self.src_table.names):
            trg_ind = names_to_trg.get(src_name, None)
            if trg_ind is not None:
                self.src_to_trg[src_ind] = trg_ind

        self.unknown_ind = names_to_trg.get('Unknown', 0)  # zero as the default unknown area

    def source_to_target(self, index):
        return self.src_to_trg.get(index, self.unknown_ind)
    
class ColorLut(object):
    """
    Class wrapper for the color lookup table.

    each column represents:
    id, name, R, G, B, A, shortname
    """

    def __init__(self, filename: os.PathLike):
        table = np.genfromtxt(os.fspath(filename), encoding="latin1", dtype=None)
        if len(table.dtype) == 6:
            # id name R G B A
            self.inds = table[table.dtype.names[0]]
            self.names = table[table.dtype.names[1]].astype('U')
            self.r = table[table.dtype.names[2]]
            self.g = table[table.dtype.names[3]]
            self.b = table[table.dtype.names[4]]
            self.a = table[table.dtype.names[5]]
            self.shortnames = np.zeros(len(self.names), dtype='U')

        elif len(table.dtype) == 7:
            # id shortname name R G B A
            self.inds = table[table.dtype.names[0]]
            self.shortnames = table[table.dtype.names[1]].astype('U')
            self.names = table[table.dtype.names[2]].astype('U')
            self.r = table[table.dtype.names[3]]
            self.g = table[table.dtype.names[4]]
            self.b = table[table.dtype.names[5]]
            self.a = table[table.dtype.names[6]]


In [32]:
fslut = ColorLut(srclutfile)
mrtlut = ColorLut(trglutfile)
regionmapping = RegionIndexMapping(srclutfile, trglutfile)

In [33]:
print(regionmapping.src_to_trg.keys())

dict_keys([0, 8, 10, 11, 12, 13, 17, 18, 26, 47, 49, 50, 51, 52, 53, 54, 58, 1001, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 2001, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035])


In [51]:
lhinds = []
rhinds = []
subcortinds = subcortstats_df.segindices

# find indices for all cortical structure
for name in lhstats_df.names:
    # find name in FS LUT File
    ind = [fslut.inds[ind] for ind, structname in enumerate(fslut.names) \
           if 'ctx-lh-'+name in structname.lower()]
    lhinds.extend(ind)
    
# find indices for all cortical structure
for name in rhstats_df.names:
    # find name in FS LUT File
    ind = [fslut.inds[ind] for ind, structname in enumerate(fslut.names) \
           if 'ctx-rh-'+name in structname.lower()]
    rhinds.extend(ind)
    
    
print(len(rhinds), len(lhinds), len(subcortinds))

34 34 45


In [55]:
srcinds = np.concatenate((rhinds, lhinds, subcortinds))

trginds = []
for ind in srcinds:
    trginds.append(regionmapping.source_to_target(ind))
    
print(trginds)

[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 0, 0, 0, 35, 36, 37, 38, 39, 0, 0, 0, 40, 41, 0, 42, 0, 0, 0, 0, 0, 0, 84, 43, 44, 45, 46, 47, 48, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [61]:
lh_volumes = create_summary_stats(lhinds, lhstats_df)

print(lh_volumes)

IndexError: index 1001 is out of bounds for axis 0 with size 34